In [111]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [112]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [113]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Oleksandr,Jędrzejewski,66083010455,Wrocław,ul. Średnia 42,48527645710,oleksandr.jedrzejewski@kuba.pl,2014-01-01,None,menadżer,71.7
1,2,Mateusz,Otworowski,79071987532,Wrocław,ul. Główna 48/36,48895355951,mateusz.otworowski@kuba.pl,2014-01-01,None,sprzedawca,45.5
2,3,Jarosław,Jasiewicz,61021421670,Wrocław,ul. Żarska 75,48050044207,jaroslaw.jasiewicz@kuba.pl,2014-01-01,None,sprzedawca,50.7
4,4,Krystyna,Morzy,62061166585,Dąbrowa Górnicza,ul. Pilipy 63/41,48068986484,krystyna.morzy@kuba.pl,2014-01-01,None,mechanik,59.8
13,5,Angelika,Wolska,89101233164,Wrocław,al. Niepodległości 33,48083603922,angelika.wolska@kuba.pl,2014-01-01,None,informatyk,58.5
3,6,Alicja,Kamińska,70061531621,Witkowo,ul. Głogowa 46/47,48860549449,alicja.kaminska@kuba.pl,2014-03-11,None,sprzedawca,47.7
5,7,Zofia,Styn,60102555246,Wrocław,ul. Zaciszna 21/44,48389895813,zofia.styn@kuba.pl,2014-04-09,2015-08-26,mechanik,48.7
6,8,Bogusława,Danesi,65032499547,Wrocław,ul. Anieli Radzickiej 28/37,48908115020,boguslawa.danesi@kuba.pl,2014-05-31,None,mechanik,45.5
7,9,Bogumiła,Nowak,65090843122,Otwock,ul. Spacerowa 6/50,48772845809,bogumila.nowak@kuba.pl,2015-05-31,None,mechanik,49.0
8,10,Łukasz,Olma,95111365034,Gorzów Wielkopolski,ul. Karpacka 14/24,48408394904,lukasz.olma@kuba.pl,2015-07-04,None,pomocnik mechanika,38.2


In [114]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [115]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [116]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [117]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [118]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [119]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [120]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [121]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [122]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [123]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Ignacy,Liuznyi,M,06290648337,Wrocław,ul. prof. Demela 24,False,48846312979,liuignacy@wp.pl
1,2,Dariusz,Szydło,M,78022874130,Wrocław,ul. Łużycka 87/36,False,48820459128,d_szydl78@wp.pl
2,3,Jan,Downar,M,85091100439,Szczytna,ul. Słonecznikowa 48/20,False,48849645589,nemgul2jtz@opayq.com
3,4,Filip,Mejna,M,85092105211,Wrocław,Aleja 1 Maja 20,False,48183073854,fil_mejn@onet.pl
4,5,Khristina,Matuszewska,K,03290548896,Legnica,ul. Leśna 41/50,True,48629770485,khr-matusze03@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1504,1505,Serhii,Piotrowicz,M,86021412435,Wrocław,ul. Targowa 95/14,False,48896155196,s_piotr@wp.pl
1505,1506,Janina,Latała,K,06261771394,Wrocław,ul. Poranna 86,False,48147973802,latala20061@vp.pl
1506,1507,Inna,Dudek,K,93020658351,Wrocław,ul. Wita Stwosza 31,False,48082877974,inn.d@gmail.com
1507,1508,Mariola,Chorąziak,K,81120135753,Wrocław,ul. Franciszka Szczepańczyka 95/26,False,48329654732,choraziak19812@gmail.com


In [124]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Ignacy,Liuznyi,M,06290648337,Wrocław,ul. prof. Demela 24,False,48846312979,liuignacy@wp.pl
1,2,Dariusz,Szydło,M,78022874130,Wrocław,ul. Łużycka 87/36,False,48820459128,d_szydl78@wp.pl
2,3,Jan,Downar,M,85091100439,Szczytna,ul. Słonecznikowa 48/20,False,48849645589,nemgul2jtz@opayq.com
3,4,Filip,Mejna,M,85092105211,Wrocław,Aleja 1 Maja 20,False,48183073854,fil_mejn@onet.pl
4,5,Khristina,Matuszewska,K,03290548896,Legnica,ul. Leśna 41/50,True,48629770485,khr-matusze03@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1504,1505,Serhii,Piotrowicz,M,86021412435,Wrocław,ul. Targowa 95/14,False,48896155196,s_piotr@wp.pl
1505,1506,Janina,Latała,K,06261771394,Wrocław,ul. Poranna 86,False,48147973802,latala20061@vp.pl
1506,1507,Inna,Dudek,K,93020658351,Wrocław,ul. Wita Stwosza 31,False,48082877974,inn.d@gmail.com
1507,1508,Mariola,Chorąziak,K,81120135753,Wrocław,ul. Franciszka Szczepańczyka 95/26,False,48329654732,choraziak19812@gmail.com


In [125]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [126]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(73)

In [127]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(87)

# auta

In [128]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [129]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [130]:
df_samochody = create_car_table(max(car_ids))

In [131]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Hyundai,iX35,4,2013,Sedan,False,inny,Benzyna,2.1,135,Automatyczna,5,5,False
1,2,Volkswagen,up!,4,2007,Sedan,False,niebieski,Benzyna,1.0,60,Manualna,5,2,False
2,3,Opel,Grandland X,4,2011,Kombi,False,biały,Benzyna,2.8,181,Manualna,5,5,False
3,4,SEAT,Ibiza,4,2012,SUV,False,złoty,Benzyna,1.1,75,Automatyczna,5,5,False
4,5,Volkswagen,Golf Plus,4,2004,SUV,False,biały,Diesel,1.5,105,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,984,Hero,Passion X pro,2,2001,,False,biały,Benzyna,0.4,59,Manualna,0,1,False
984,985,Volkswagen,Golf Sportsvan,4,2008,Sedan,False,srebrny,Benzyna,2.0,131,Manualna,5,5,False
985,986,Fiat,500,4,2010,Coupe,False,czarny,Benzyna,1.0,69,Manualna,3,2,False
986,987,Volkswagen,Polo,4,2003,Hatchback,False,szary,Benzyna,1.4,95,Manualna,5,5,True


In [132]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [133]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna'], dtype=object)

# Usługi

In [134]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [135]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [136]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [137]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_20636\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2009' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [138]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [139]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,95500.0
1,2,2,2,2,2014-01-04,17800.0
2,3,3,3,6,2014-01-04,84800.0
3,4,4,4,3,2014-01-04,40100.0
4,5,5,5,2,2014-01-07,27500.0
...,...,...,...,...,...,...
925,926,985,1501,2,2020-12-18,30000.0
926,927,986,1363,6,2020-12-26,24200.0
927,928,217,1425,3,2020-12-27,163000.0
928,929,987,1508,2,2020-12-31,21300.0


In [140]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [141]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [142]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():
        sell_prices[i] *= 0.97

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_20636\1097819302.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():


[np.int64(12), np.int64(1), np.int64(1), np.int64(6), np.int64(1), np.int64(6), np.int64(1), np.int64(6), np.int64(6), np.int64(1), np.int64(6), np.int64(1), np.int64(12), np.int64(1), np.int64(6), np.int64(1), np.int64(18), np.int64(18), np.int64(6), np.int64(12), np.int64(1), np.int64(1), np.int64(18), np.int64(1), np.int64(12), np.int64(1), np.int64(12), np.int64(6), np.int64(12), np.int64(1), np.int64(18), np.int64(1), np.int64(1), np.int64(12), np.int64(6), np.int64(1), np.int64(1), np.int64(6), np.int64(6), np.int64(1), np.int64(1), np.int64(18), np.int64(18), np.int64(12), np.int64(12), np.int64(12), np.int64(6), np.int64(18), np.int64(18), np.int64(1), np.int64(1), np.int64(1), np.int64(6), np.int64(18), np.int64(1), np.int64(18), np.int64(1), np.int64(6), np.int64(18), np.int64(1), np.int64(18), np.int64(6), np.int64(1), np.int64(18), np.int64(12), np.int64(18), np.int64(6), np.int64(1), np.int64(1), np.int64(6), np.int64(6), np.int64(1), np.int64(1), np.int64(12), np.int64(6)

In [143]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,8,21,6,95264.0,12,2014-01-26
1,2,5,22,6,34500.0,1,2014-01-26
2,3,13,23,2,40600.0,1,2014-01-26
3,4,3,24,3,105774.0,6,2014-01-26
4,5,15,25,6,52100.0,1,2014-01-30
...,...,...,...,...,...,...,...
920,921,985,70,3,39479.0,1,2020-12-24
921,922,978,1504,6,102306.0,6,2020-12-27
922,923,983,1505,3,60282.0,6,2020-12-27
923,924,986,1506,6,34840.0,12,2020-12-27


In [144]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [145]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [146]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(23), array([56])), (np.int64(26), array([33])), (np.int64(40), array([0])), (np.int64(41), array([31])), (np.int64(53), array([9])), (np.int64(60), array([32])), (np.int64(64), array([57])), (np.int64(66), array([53])), (np.int64(88), array([34])), (np.int64(92), array([13])), (np.int64(101), array([11,  6])), (np.int64(107), array([22])), (np.int64(110), array([46])), (np.int64(127), array([7])), (np.int64(129), array([21, 41, 46])), (np.int64(139), array([22])), (np.int64(156), array([61])), (np.int64(160), array([14])), (np.int64(161), array([36, 43])), (np.int64(167), array([5])), (np.int64(170), array([39])), (np.int64(175), array([19])), (np.int64(185), array([30])), (np.int64(188), array([36])), (np.int64(193), array([36])), (np.int64(203), array([40])), (np.int64(214), array([24])), (np.int64(215), array([39])), (np.int64(216), array([58])), (np.int64(217), array([54])), (np.int64(224), array([26])), (np.int64(225), array([6])), (np.int64(229), array([26, 52, 49])), 

In [147]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [148]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Kierownica,Bajaj Pulsar 180,32,1,1
1,Fotel tylny,Kia Picanto skóra,168,2,1
2,Maska,Audi S5 biały,1913,3,1
3,Zagłówek,Opel Insignia,26,4,2
4,Drzwi prawe przednie,Mercedes-Benz V 300 czarny,873,5,1
...,...,...,...,...,...
201,Wał sterujący,,150,142,1
202,Fotel przedni,Ford Ka/Ka+ materiał,200,143,1
203,Rozrusznik,SEAT Ibiza,361,144,1
204,Drzwi lewe przednie,SEAT Ibiza szary,510,144,1


# Wyposażenie

In [149]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [150]:
df_equipment

,id,nazwa,atrybut
0,1,Kierownica,Bajaj Pulsar 180
1,2,Fotel tylny,Kia Picanto skóra
2,3,Maska,Audi S5 biały
3,4,Zagłówek,Opel Insignia
4,5,Drzwi prawe przednie,Mercedes-Benz V 300 czarny
...,...,...,...
200,154,Fotel tylny,MINI Cooper S Countryman materiał
202,155,Fotel przedni,Ford Ka/Ka+ materiał
203,156,Rozrusznik,SEAT Ibiza
204,157,Drzwi lewe przednie,SEAT Ibiza szary


In [151]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi - uzupełnienie

In [152]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [153]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,23,44,9,2014-02-19,40
1,2,26,49,9,2014-02-25,190
2,3,40,76,7,2014-04-01,2180
3,4,41,77,8,2014-04-02,60
4,5,53,101,7,2014-05-09,990
...,...,...,...,...,...,...
140,141,945,1453,7,2020-08-26,440
141,142,956,1466,7,2020-09-25,170
142,143,957,1468,4,2020-09-25,230
143,144,962,1472,7,2020-10-18,990


In [154]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [155]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [156]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,2,2,1
2,3,3,3,1
3,4,4,4,2
4,5,5,5,1
...,...,...,...,...
201,202,142,64,1
202,203,143,155,1
203,204,144,156,1
204,205,144,157,1


In [157]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [158]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


KeyError: 'cena_części'

In [ ]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,665.0,2.0
1,2.0,2.0,110.0,6.0
2,3.0,3.0,4420.0,11.0
3,4.0,4.0,493.0,2.0
4,5.0,5.0,520.0,1.0
...,...,...,...,...
132,133.0,133.0,1442.0,6.0
133,134.0,134.0,664.0,1.0
134,135.0,135.0,148.0,7.0
135,136.0,136.0,259.0,5.0


In [ ]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]